In [1]:
import shutil
import os
from behavioural_benchmark.indicators import MemoisedIndicators

### IEEE AMMA: Indicator calculation

Here I calculate the indicators from a path containing folders with my experiment results. This also involves moving the processed results to make my life easier.

In [2]:
current_dir = os.path.dirname(os.getcwd())
fresh_results = os.path.join(current_dir, '../fresh_results')
if not os.path.exists(fresh_results):
    os.makedirs(fresh_results)
result_path = os.path.join(current_dir, "../Results")
if not os.path.exists(result_path):
    os.makedirs(result_path)

In [5]:
def get_tags(tag_path):
    """
    Given a path to a folder like 'experiment/dimensions/population/total_iterations/function_name/metaheuristic/parameters/run',
    split these into tags 
    :param tag_path: the path to where the results are stored
    :return: the tags for the experiment
    """
    experiment, dimension, population, total_iterations, function_name, metaheuristic, parameters, run = tag_path.split("/")[-8:]
    return experiment, dimension, population, total_iterations, function_name, metaheuristic, control_parameters, run

def write_results(i: MemoisedIndicators, write_path, write_tags):
    DRoC_A, DRoC_B, ERT_Diversity, Critical_Diversity = i.get_DRoC_A(), i.get_DRoC_B(), i.get_ERT_Diversity(), i.get_Critical_Diversity()
    FRoC_A, FRoC_B, ERT_Fitness, Critical_Fitness = i.get_FRoC_A(), i.get_FRoC_B(), i.get_ERT_Fitness(), i.get_Critical_Fitness()
    SRoC_A, SRoC_B, ERT_Separation, Critical_Separation = i.get_SRoC_A(), i.get_SRoC_B(), i.get_ERT_Separation(), i.get_Critical_Separation()
    MRoC_A, MRoC_B, ERT_Mobility, Critical_Mobility = i.get_MRoC_A(), i.get_MRoC_B(), i.get_ERT_Mobility(), i.get_Critical_Mobility()
    ntotal, nbest, nshared, best_strength = i.get_ntotal(), i.get_nbest(), i.get_nshared(), i.get_best_strength()
    MID, MGC, SNID = i.get_MID(), i.get_MGC(), i.get_SNID()
    EXPLORE, INFEASIBLE = i.get_EXPLORE_Percent(), i.get_INFEASIBLE_Percent()

    if not os.path.exists(write_path):
        os.makedirs(write_path)
    with open(f"{write_path}/results.csv", 'a+') as f:
        experiment, dimension, pop_size, max_iteration, function, alg, control_params, run = write_tags
        f.write( # 34
            f"{experiment},{dimension},{pop_size},{max_iteration},{function},{alg},{control_params},{run}," #8
            f"{DRoC_A},{DRoC_B},{ERT_Diversity},{Critical_Diversity}," #4
            f"{FRoC_A},{FRoC_B},{ERT_Fitness},{Critical_Fitness}," #4
            f"{SRoC_A},{SRoC_B},{ERT_Separation},{Critical_Separation}," #4
            f"{MRoC_A},{MRoC_B},{ERT_Mobility},{Critical_Mobility}," #4
            f"{ntotal},{nbest},{nshared},{best_strength}," #4
            f"{MID},{MGC},{SNID}," #3
            f"{EXPLORE},{INFEASIBLE}\n") #3

def clean(src):
    roots = [dir_root for dir_root, _, _ in os.walk(src)]
    roots.reverse()
    for dir_root in roots:
        if os.path.exists(dir_root) and len(os.listdir(dir_root)) == 0:
            os.rmdir(dir_root)

In [6]:
expected_experiment_name = "characteristicTest"
for experiment_name in os.listdir(fresh_results):
    if expected_experiment_name not in experiment_name: continue
    for dim in os.listdir(f"{fresh_results}/{experiment_name}"):
        for population_size in os.listdir(f"{fresh_results}/{experiment_name}/{dim}"):
            for max_iter in os.listdir(f"{fresh_results}/{experiment_name}/{dim}/{population_size}"):
                for problem in os.listdir(f"{fresh_results}/{experiment_name}/{dim}/{population_size}/{max_iter}"):
                    problem_root = f"{fresh_results}/{experiment_name}/{dim}/{population_size}/{max_iter}/{problem}"
                    for algorithm in os.listdir(problem_root):
                        for control_parameters in os.listdir(f"{problem_root}/{algorithm}"):
                            root = f"{problem_root}/{algorithm}/{control_parameters}"
                            if not os.path.isdir(root): continue
                            for run in os.listdir(root):
                                input_arg = f"{root}/{run}"
                                indicators = MemoisedIndicators(input_arg)
                                tags = get_tags(indicators.path)
                                write_results(i=indicators, write_path=f"{result_path}/{experiment_name}/", write_tags=tags)
                                shutil.move(
                                    f"{root}/{run}",
                                    f"{root}/{run}".replace("fresh_results", "processed_results")
                                    )
    clean(f"{fresh_results}/{experiment_name}")

/home/lauren/git/code_masters/behaviouralBenchmarking/.venv2/lib/python3.9/site-packages/pwlf/pwlf.py:1113: RuntimeWarning: invalid value encountered in divide
  self.slopes = np.divide(
